In [ ]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.3 MB/s eta 0:00:00


In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install llama-cpp-python

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.7-cp311-cp311-linux_x86_64.whl size=4552832 sha256=d95d4e28fe01861ca21fc0ae5e2508da18ead24586b9427e7a9779bd72558eae
  Stored in directory: /root/.cache/pip/wheels/eb/82/79/ac77fcd49324b75ae6aa18e63a87cf9da4371a57e2cdc8dc03
Successfully built llama-cpp-python


In [ ]:
import os
from typing import Dict, TypedDict, List, Any
from llama_cpp import Llama
from langchain.tools import TavilySearchResults
from langgraph.graph import StateGraph, END

os.environ["TAVILY_API_KEY"] = "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"

In [ ]:
llm = Llama.from_pretrained(
    repo_id="soykot2910/mistral_model_ollama",
    filename="unsloth.F16.gguf",
    n_ctx=2048,
    verbose=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


unsloth.F16.gguf:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


In [ ]:
class AgentState(TypedDict):
    query: str
    research_data: List[Dict[str, Any]]
    final_answer: str

def research_agent(state):
    query = state["query"]
    search_tool = TavilySearchResults(max_results=3)
    results = search_tool.invoke(query)

    formatted_results = []
    for result in results:
        formatted_results.append({
            "title": result.get("title", ""),
            "content": result.get("content", "")[:200] + "..."
        })

    return {"research_data": formatted_results}

def answer_agent(state):
    research_data = state["research_data"]
    query = state["query"]
    prompt = f"""[INST] You are a research assistant. Answer based on this data:
{research_data}
Question: {query}
Please provide a concise answer. [/INST]"""

    response = llm(
        prompt,
        max_tokens=256,
        temperature=0.7
    )

    answer = response["choices"][0]["text"]
    return {"final_answer": answer}

workflow = StateGraph(AgentState)
workflow.add_node("research", research_agent)
workflow.add_node("answer", answer_agent)

workflow.set_entry_point("research")
workflow.add_edge("research", "answer")
workflow.add_edge("answer", END)

graph = workflow.compile()

def run_research_system(query):
    inputs = {"query": query}
    try:
        result = graph.invoke(inputs)
        return result["final_answer"]
    except Exception as e:
        return f"Error: {str(e)}"

if __name__ == "__main__":
    query = "Latest advancements in AI research"
    answer = run_research_system(query)
    print("\nFinal Answer:\n", answer)


Final Answer:
  The latest advancements in AI research as per the provided data include:

1. Development of a novel 6D pose dataset for improving AI capabilities, particularly in areas like robotics and computer vision.
2. Advancements in Generative AI Development, which is a subfield of AI that focuses on creating new data instances similar to the given data, such as generating human-like text, images, or music.
3. Improvements in deep learning in natural language processing, which has led to better optimization in systems and control engineering.
